In [ ]:
!pip install kaggle

In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

ref                                                    title                                        size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------  ------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
thedrcat/daigt-v2-train-dataset                        DAIGT V2 Train Dataset                       29MB  2023-11-16 01:38:36           2907        246  1.0              
thedevastator/global-video-game-sales-and-reviews      Global Video Game Sales and Reviews          56KB  2023-12-20 06:36:51           1087         30  1.0              
bhavikjikadara/car-price-prediction-dataset            Car Price Prediction Dataset                  4KB  2023-12-23 07:07:54           1090         31  0.9411765        
andreinovikov/air-pollution                            Air pollution                                96KB  2023-12-05 12:00:34           1103     

In [ ]:
!kaggle datasets download -d arbethi/diabetic-retinopathy-level-detection

100% 9.65G/9.66G [01:42<00:00, 165MB/s]
100% 9.66G/9.66G [01:42<00:00, 101MB/s]


In [ ]:
!unzip -q /content/diabetic-retinopathy-level-detection.zip

In [ ]:
imageSize = [299, 299]
trainPath = r'/content/preprocessed dataset/preprocessed dataset/train'
testPath = r'/content/preprocessed dataset/preprocessed dataset/test'

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.xception import Xception, preprocess_input
from glob import glob
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('/content/preprocessed dataset/preprocessed dataset/training',
                                      target_size=(299, 299),
                                      class_mode='categorical',
                                      batch_size=32)
test_set = test_datagen.flow_from_directory('/content/preprocessed dataset/preprocessed dataset/testing',
                                    target_size=(299, 299),
                                      class_mode='categorical',
                                      batch_size=32)

Found 3662 images belonging to 5 classes.
Found 734 images belonging to 5 classes.


In [ ]:
training_set.class_indices

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4}

In [ ]:
xception = Xception(input_shape=imageSize + [3], weights='imagenet', include_top=False)

83683744/83683744 [==============================] - 0s 0us/step


In [ ]:
for layer in xception.layers:
  layer.trainable = False

In [ ]:
x = Flatten()(xception.output)

In [ ]:
prediction = Dense(5, activation='softmax')(x)

In [ ]:
model = Model(inputs=xception.input, outputs=prediction)

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
batch_size = 32
steps_per_epoch = len(training_set) // batch_size

In [ ]:
model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=30,
  steps_per_epoch=len(training_set) // batch_size,

  validation_steps=len(test_set)
)

<ipython-input-24-b8d67da5f866>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/30
3/3 [==============================] - 484s 229s/step - loss: 7.5395 - accuracy: 0.4271 - val_loss: 12.0556 - val_accuracy: 0.3828
Epoch 2/30
3/3 [==============================] - 476s 226s/step - loss: 11.1324 - accuracy: 0.4479 - val_loss: 7.8777 - val_accuracy: 0.5627
Epoch 3/30
3/3 [==============================] - 503s 240s/step - loss: 7.8235 - accuracy: 0.5938 - val_loss: 7.6342 - val_accuracy: 0.6798
Epoch 4/30
3/3 [==============================] - 475s 227s/step - loss: 7.9031 - accuracy: 0.6667 - val_loss: 6.2273 - val_accuracy: 0.6567
Epoch 5/30
3/3 [==============================] - 503s 239s/step - loss: 11.2066 - accuracy: 0.4479 - val_loss: 7.6107 - val_accuracy: 0.5150
Epoch 6/30
3/3 [==============================] - 501s 239s/step - loss: 5.2464 - accuracy: 0.6042 - val_loss: 6.3114 - val_accuracy: 0.5777
Epoch 7/30
3/3 [==============================] - 451s 214s/step - loss: 4.0950 - accuracy: 0.7083 - val_loss: 6.4169 - val_accuracy: 0.6676
Epoch 8/30

In [ ]:
model.save('Colab_Model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
